In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import torch
import numpy as np
from skimage.metrics import structural_similarity as ssim
import torch.nn.modules as nn
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # Encoder: Increased capacity with higher channel counts
        self.encoder1 = self.conv_block(3, 64)
        self.encoder2 = self.conv_block(64, 128)
        self.encoder3 = self.conv_block(128, 256)
        self.encoder4 = self.conv_block(256, 512)
        self.bottleneck = self.conv_block(512, 1024)

        # Decoder
        self.upconv4 = self.upconv(1024, 512)
        self.decoder4 = self.conv_block(1024, 512)
        self.upconv3 = self.upconv(512, 256)
        self.decoder3 = self.conv_block(512, 256)
        self.upconv2 = self.upconv(256, 128)
        self.decoder2 = self.conv_block(256, 128)
        self.upconv1 = self.upconv(128, 64)
        self.decoder1 = self.conv_block(128, 64)

        # Final output layer
        self.final = nn.Sequential(
            nn.Conv2d(64, 1, kernel_size=1),
            nn.Sigmoid()
        )

        # Max-pooling for down-sampling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def conv_block(self, in_channels, out_channels):
        """Convolutional block: Two conv layers with BatchNorm and ReLU."""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def upconv(self, in_channels, out_channels):
        """Transposed convolution for up-sampling."""
        return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x):
        # Encoder
        enc1 = self.encoder1(x)  # Output: 64 channels
        enc2 = self.encoder2(self.pool(enc1))  # Output: 128 channels
        enc3 = self.encoder3(self.pool(enc2))  # Output: 256 channels
        enc4 = self.encoder4(self.pool(enc3))  # Output: 512 channels

        # Bottleneck
        bottleneck = self.bottleneck(self.pool(enc4))  # Output: 1024 channels

        # Decoder
        dec4 = self.decoder4(torch.cat((self.upconv4(bottleneck), enc4), dim=1))  # Output: 512 channels
        dec3 = self.decoder3(torch.cat((self.upconv3(dec4), enc3), dim=1))  # Output: 256 channels
        dec2 = self.decoder2(torch.cat((self.upconv2(dec3), enc2), dim=1))  # Output: 128 channels
        dec1 = self.decoder1(torch.cat((self.upconv1(dec2), enc1), dim=1))  # Output: 64 channels

        # Final depth map
        out = self.final(dec1)
        return out

# Predict Video

In [3]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
def temporal_consistency(pred_frame_t, pred_frame_t1):
    """
    Computes temporal consistency (MSE) between consecutive predicted frames.

    Args:
        pred_frame_t (torch.Tensor): Predicted depth map at time t [1, H, W].
        pred_frame_t1 (torch.Tensor): Predicted depth map at time t+1 [1, H, W].

    Returns:
        float: Temporal consistency loss (MSE).
    """
    return torch.nn.functional.mse_loss(pred_frame_t, pred_frame_t1).item()

# Visualization function
def visualize_comparison(rgb_frame, pred_depth1, pred_depth2, frame_idx):
    """
    Visualize the comparison between two predicted depth maps.

    Args:
        rgb_frame (np.array): Original RGB frame.
        pred_depth1 (np.array): Predicted depth map from model 1.
        pred_depth2 (np.array): Predicted depth map from model 2.
        frame_idx (int): Current frame index.
    """
    pred_depth2 /= 10
    plt.figure(figsize=(12, 6))

    # Show input frame
    plt.subplot(1, 3, 1)
    plt.imshow(rgb_frame)
    plt.title(f"Frame {frame_idx}")
    plt.axis("off")

    # Show prediction from model 1
    plt.subplot(1, 3, 2)
    plt.imshow(pred_depth1, cmap='plasma')
    plt.title("Model 1 Prediction")
    plt.axis("off")

    # Show prediction from model 2
    plt.subplot(1, 3, 3)
    plt.imshow(pred_depth2, cmap='plasma')
    plt.title("Model 2 Prediction")
    plt.axis("off")

    # Save or show frame
    plt.savefig(f"comparison_frame_{frame_idx}.png")
    plt.close()

In [4]:
# Compare two models on a video
def compare_depth_models_no_ground_truth(model1, model2, video_path, device="cuda"):
    """
    Compares two depth estimation models on a video without ground truth.

    Args:
        model1 (torch.nn.Module): First depth estimation model.
        model2 (torch.nn.Module): Second depth estimation model.
        video_path (str): Path to the input video.
        device (str): Device to run the models on.

    Returns:
        dict: Temporal consistency metrics for both models.
    """
    # Initialize metrics
    temporal_metrics_model1 = []
    temporal_metrics_model2 = []

    # Open video file
    cap = cv2.VideoCapture(video_path)
    prev_depth1, prev_depth2 = None, None
    frame_idx = 0
    i = 0
    # Process each frame
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        print(i)
        i+=1

        # Preprocess frame (convert to tensor and normalize)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_frame_tensor = torch.from_numpy(rgb_frame).float().permute(2, 0, 1).unsqueeze(0).to(device) / 255.0

        # Predict depth using both models
        pred_depth1 = model1(rgb_frame_tensor)
        pred_depth2 = model2(rgb_frame_tensor)

        # Resize predictions to match input frame
        pred_depth1 = torch.nn.functional.interpolate(pred_depth1, size=rgb_frame_tensor.shape[-2:], mode='bilinear', align_corners=False)
        pred_depth2 = torch.nn.functional.interpolate(pred_depth2, size=rgb_frame_tensor.shape[-2:], mode='bilinear', align_corners=False)

        # Compute temporal consistency if not the first frame
        if prev_depth1 is not None and prev_depth2 is not None:
            temporal_metrics_model1.append(temporal_consistency(prev_depth1, pred_depth1))
            temporal_metrics_model2.append(temporal_consistency(prev_depth2, pred_depth2))

        # Update previous frame predictions
        prev_depth1 = pred_depth1.clone()
        prev_depth2 = pred_depth2.clone()

        # Visualize results (Optional)
        visualize_comparison(rgb_frame, pred_depth1[0, 0].detach().cpu().numpy(), pred_depth2[0, 0].detach().cpu().numpy(), frame_idx)

        frame_idx += 1

    # Release video capture
    cap.release()

    # Compute average temporal consistency
    avg_temporal_model1 = np.mean(temporal_metrics_model1)
    avg_temporal_model2 = np.mean(temporal_metrics_model2)

    return {"Model1_Temporal": avg_temporal_model1, "Model2_Temporal": avg_temporal_model2}

In [5]:
class UNet_old(nn.Module):
    def __init__(self):
        super(UNet_old, self).__init__()
        # encoder
        self.encoder1 = self.conv_block(3, 32)
        self.encoder2 = self.conv_block(32, 64)
        self.encoder3 = self.conv_block(64, 128)
        self.encoder4 = self.conv_block(128, 256)
        # decoder
        self.decoder1 = self.conv_block(256, 128)
        self.decoder2 = self.conv_block(128, 64)
        self.decoder3 = self.conv_block(64, 32)
        # one chnnel for depth estimation
        self.final = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.Conv2d(16, 1, kernel_size=3, padding=1),
        )

        # up-sample and down-sample
        self.pool = nn.MaxPool2d(2, 2)
        self.up1 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.up2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.up3 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool(enc1))
        enc3 = self.encoder3(self.pool(enc2))

        enc4 = self.encoder4(self.pool(enc3))

        dec1 = self.decoder1(torch.cat((self.up1(enc4), enc3), dim=1))
        dec2 = self.decoder2(torch.cat((self.up2(dec1), enc2), dim=1))
        dec3 = self.decoder3(torch.cat((self.up3(dec2), enc1), dim=1))
        out = self.final(dec3)

        return out

In [6]:
# Load models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model1 = UNet_old()
checkpoint = torch.load("/content/gdrive/MyDrive/ECE_1508_project/epoch_149.pth")
model1.load_state_dict(checkpoint)
model1.to(device)
model1.eval()

model2 = UNet()
checkpoint = torch.load("/content/gdrive/MyDrive/ECE_1508_project/supervised_edge_aware_temporal_e_30.pth")
model2.load_state_dict(checkpoint)
model2.to(device)
model2.eval()
# Path to the video
video_path = "/content/gdrive/MyDrive/ECE_1508_project/test1.mp4"


<ipython-input-6-b102a171f70f>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/content/gdrive/MyDrive/ECE_1508_project/epoch_149.pth")
<ipython-in

In [7]:
# Compare models
metrics = compare_depth_models_no_ground_truth(model1, model2, video_path, device="cuda")
print("Temporal Consistency Metrics:")
print(metrics)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27